<a href="https://colab.research.google.com/github/The7redarrow/ml-class/blob/master/Copy_of_Working_version_of_bagging_n_pasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bagging and Pasting

#### Part of the [Inquiryum Machine Learning Fundamentals Course](http://inquiryum.com/machine-learning/)

![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/divider.png)

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bagging..png)


Now we are about to embark on our journey from simple decision trees to algorithms that use decision trees as components. The path goes like this:


![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/dbxg.png)

The use of decision trees began in the 1980s and XGBoost was introduced in 2016. Throughout the next few notebooks we will explore this progression of algorithms.  

### A collective of classifiers

To gain an intuition on how this works, let's look at how our confidence might increase when more people tell us something. Whether it is multiple doctors giving us the same diagnosis or something as simple as ...

#### The Mary Spender example

Let's say one of your friends mentions over lunch that you would love a particular musical artist on YouTube, say Mary Spender, who you never heard before. 

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/MarySpender2.png)

What is the chance that you will actually like Mary Spender's music? Maybe slighly better than chance? Let's say you think there is a 60% chance you will like her. You will file away the recommendation but you are not going to rush home and watch a YouTube video.  Now, in addition to the lunch friend's recommendation,  an old music school friend, now living in Austin messages you saying you should check out Mary Spender and the friend predicts you will absolutely love her. Then a week later, while talking with an old bandmate over the phone, that bandmate, again, recommends Mary Spender. Over the course of less than 10 days, three of your friends independently (because they don't know one another) recommend Mary Spender. Now what is the likelihood of you liking Mary Spender? I am guessing you think that now it is higher than 60%. Maybe now you think it is 90% likely you will like her. It is the aggregate of these 3 people's opinions (3 classifiers) that ups the accuracy of the prediction.

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/spender22.png)


This is similar to how bagging works. One aggregates the votes of a number of classifiers and the vote of that ensemble of classifiers is more accurate than that of a single classifier. Even if the accuracy of each component classifier is low (known as a weak classifier), the ensemble can be a strong (high accuracy) classifier. Of course there are some caveats. 

Back to the Mary Spender example. Suppose one of your friends went to a Mary Spender concert and then later in the week met with four other of your friends and mentioned that she thought you would love Mary Spender's music. Then, over the course of a week all those friends recommended Mary Spender to you. In this case the recommendations are not that independent---all are based on one person's opinion. Thus, the accuracy would not be as great as in the example above. Similarly, if you made 10 copies of the exact same classifier each trained on exactly the same data, the accuracy of the ensemble of clones would not be any better than the accuracy of a single copy. Moving away from Mary Spender and our musical tastes and back to machine learning, we can try to create independence among the classifier in 2 ways:

1. We can change the type of classifier. For example, we can use a k-Nearest Neighbor Classifier with Manhattan distance and a k of 5, a k-Nearest Neighbor Classifier with Euclidean distance and a k of 3, a decision tree classifier using entropy and a max depth of 5, and a decision tree classifier with using gini and no max depth specified. Hopefully, the accuracy of the ensemble of the four classifiers would be greater than that of a single classifier.
2. We can have an ensemble of the same classifier (for example, 10 decision tree classifiers with identical hyperparameters) but each classifier can get a different subset of the training data. The classifiers would thus build different models (differents 'rules') and, again, the accuracy of the ensemble should be greather than that of a single classifier. This is the approach we will take.

### Bagging and Pasting


In this Jupyter notebook, we are going to explore Bagging algorithms. Bagging algorithms come in a variety of 'flavors' including one called 'bagging' and one called 'pasting'.

But first an experiment on the what *with replacement* means. As you will see shortly, that term is the crucial difference between bagging and pasting.

### A small experiment
NOTE: The following code is just used for illustration and is nothing we will be using for machine learning. 

Consider a list of 5 red balls and 5 blue balls:

In [22]:
bag = ['red', 'red', 'red', 'red', 'red',
       'blue', 'blue', 'blue', 'blue', 'blue']

Suppose we want to pick 7 random balls from this list. Python offers two functions that will give us random elements from a list.One is called `choices` which selects a sample with replacement, which means that once a ball is selected it is put back in the bag so it has the potential to be selected again. Let's give it a try, and just because things are random let's do this 100 times:

In [23]:
import random
total = 0
for i in range(100):
    set = random.choices(bag, k=7)
    blue = set.count('blue')
    red = set.count('red')
    if blue > 5 or red > 5:
        print("%i blue and %i red" % (set.count('blue'), set.count('red')))
        total +=1
print("Balls selected exceeded balls in bag: %i" % (total))

1 blue and 6 red
1 blue and 6 red
1 blue and 6 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
6 blue and 1 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
1 blue and 6 red
1 blue and 6 red
1 blue and 6 red
Balls selected exceeded balls in bag: 13


*A reminder: please don't mindlessly execute the code. Look at it and understand it*

There are five blue balls. Since we are doing the selection with replacement there are times when we select more than 5 blue balls (or five red ones).  

When we print

```
Balls selected exceeded balls in bag: 13
```

it shows how many times that was the case.


When I ran this, 14 times out of 100 had more of one color ball than there were in the original bag. In fact, several times I ended up with all 7 of the balls blue, even though the original list had only 5 balls:

```
7 blue and 0 red
6 blue and 1 red
7 blue and 0 red
1 blue and 6 red
0 blue and 7 red
1 blue and 6 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
1 blue and 6 red
6 blue and 1 red
6 blue and 1 red
Balls selected exceeded balls in bag: 14
```
Again, this is called selecting with replacement (we put what we selected back in the set before selecting again). 

The other alternative is to select without replacement--once we select something we can't select it again. Python's `sample` does this:


In [24]:
import random
total = 0
for i in range(1000):
    set = random.sample(bag, k=7)
    blue = set.count('blue')
    red = set.count('red')
    if blue > 5 or red > 5:
        print("%i blue and %i red" % (set.count('blue'), set.count('red')))
        total +=1
print("Balls selected exceeded balls in bag: %i" % (total))

Balls selected exceeded balls in bag: 0


As you can see, the number of a specific colored ball that we select never exceeded the number of balls of that color in the original set.

Now back to bagging and pasting. In both approaches we are going to sample the training data. Let's say we want 70% of the training data in our sample. In bagging ([Breiman, 1996](https://link.springer.com/content/pdf/10.1007/BF00058655.pdf)), if we our training dataset is 1000 instances and we want 70% for a particular classifier, the algorithm will randomly select 700 out of the 1,000 **with replacement**. With pasting ([Breiman, 1998](https://link.springer.com/article/10.1023/A:1007563306331)), the selection is done **without replacement**. 

#### but wait, there is more ...

There are two other options. Instead of selecting a random subset of training data instances, we can select a random subset of columns (features). Let's say we have a dataset of 1,000 instances each with 100 features. When we select a random subset of columns, we still have 1,000 instances but now they have just a subset of the features. This is called Random Subspaces ([Ho, 1998](https://pdfs.semanticscholar.org/b41d/0fa5fdaadd47fc882d3db04277d03fb21832.pdf?_ga=2.196949164.1638238666.1596910000-1073138517.1596910000)).

Finally, we can train a classifier on both random subsets of instances and random subsets of features. This is known as Random Patches ([Louppe and Geurts, 2012](https://www.researchgate.net/publication/262212941_Ensembles_on_Random_Patches))

In summary, the four methods are:

* **bagging** - select a subset of data set instances using replacement
* **pasting** - select a subset of data set instances without replacement
* **Random Subspaces** - select a subset of features
* **Random Patches** - select both a subset of features and of instances

Let's see how this works!

First, let's grab the Wisconsin Cancer data we used before:

#### Wisconsin Cancer Dataset
![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/aimam.png)
image from Nvidia's [AI Improves Breast Cancer Diagnoses by Factoring Out False Positives](https://blogs.nvidia.com/blog/2018/02/01/making-mammography-more-meaningful/)

[A description of the Cancer Database](#Breast-Cancer-Database)

In this dataset we are trying to predict the diagnosis---either M (malignant) or B (benign).

Let's load the dataset and split it into training and testing sets

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

colNames = ['id', 'diagnosis', 'radiusAvg', 'textureAvg', 'perimeterAvg', 'areaAvg',
            'smoothnessAvg', 'compactnessAvg', 'concavityAvg', 'concavityPointsAvg',
            'symmetryAvg', 'FractalDimensionAvg', 'radiusSE', 'textureSE', 'perimeterSE',
            'areaSE','smoothnessSE', 'compactnessSE', 'concavitySE', 'concavityPointsSE',
            'symmetrySE', 'FractalDimensionSE', 'radiusWorst', 'textureWorst', 'perimeterWorst',
            'areaWorst', 'smoothnessWorst', 'compactnessWorst', 'concavityWorst', 'concavityPointsWorst',
            'symmetryWorst>', 'FractalDimensionWorst']
len(colNames)

data = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/wdbc.data', names=colNames)
data.set_index('id', inplace=True)

trainingdata, testdata = train_test_split(data, test_size = 0.2)
testdata

,diagnosis,radiusAvg,textureAvg,perimeterAvg,areaAvg,smoothnessAvg,compactnessAvg,concavityAvg,concavityPointsAvg,symmetryAvg,...,radiusWorst,textureWorst,perimeterWorst,areaWorst,smoothnessWorst,compactnessWorst,concavityWorst,concavityPointsWorst,symmetryWorst>,FractalDimensionWorst
id,,,,,,,,,,,,,,,,,,,,,
89812,M,23.51,24.27,155.10,1747.0,0.10690,0.12830,0.23080,0.141000,0.1797,...,30.67,30.73,202.40,2906.0,0.15150,0.26780,0.48190,0.20890,0.2593,0.07738
925291,B,11.51,23.93,74.52,403.5,0.09261,0.10210,0.11120,0.041050,0.1388,...,12.48,37.16,82.28,474.2,0.12980,0.25170,0.36300,0.09653,0.2112,0.08732
89382601,B,14.61,15.69,92.68,664.9,0.07618,0.03515,0.01447,0.018770,0.1632,...,16.46,21.75,103.70,840.8,0.10110,0.07087,0.04746,0.05813,0.2530,0.05695
871641,B,11.08,14.71,70.21,372.7,0.10060,0.05743,0.02363,0.025830,0.1566,...,11.35,16.82,72.01,396.5,0.12160,0.08240,0.03938,0.04306,0.1902,0.07313
8910748,B,11.29,13.04,72.23,388.0,0.09834,0.07608,0.03265,0.027550,0.1769,...,12.32,16.18,78.27,457.5,0.13580,0.15070,0.12750,0.08750,0.2733,0.08022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873885,M,15.28,22.41,98.92,710.6,0.09057,0.10520,0.05375,0.032630,0.1727,...,17.80,28.03,113.80,973.1,0.13010,0.32990,0.36300,0.12260,0.3175,0.09772
9113455,B,13.14,20.74,85.98,536.9,0.08675,0.10890,0.10850,0.035100,0.1562,...,14.80,25.46,100.90,689.1,0.13510,0.35490,0.45040,0.11810,0.2563,0.08174
919537,B,10.96,17.62,70.79,365.6,0.09687,0.09752,0.05263,0.027880,0.1619,...,11.62,26.51,76.43,407.5,0.14280,0.25100,0.21230,0.09861,0.2289,0.08278


Now divide up the data into the features and labels

In [26]:
colNames.remove('id')
colNames.remove('diagnosis')
trainingDataFeatures = trainingdata[colNames]
testDataFeatures = testdata[colNames]
trainingDataLabels = trainingdata['diagnosis']
testDataLabels = testdata['diagnosis']


Let's get a base accuracy using a single decision tree classifier:

In [27]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = clf.predict(testDataFeatures)

accuracy_score(testDataLabels, predictions)

0.9210526315789473

Now we will see if we can improve on that accuracy.

### Building a bagging classifier

Let's build a collective of 20 decision tree classifiers (`n_estimators`). Let's train each one with 100 random samples from our dataset (`max_samples`) with replacement (`bootstrap=True`). `n_jobs` means how many jobs to run in parallel. `n_jobs=-1` means use all available CPU cores.   

Just to reinforce the vocabulary we are learning, `n_estimators`, `max_samples`, `bootstrap` are among the **hyperparameters** of the bagging classifier.

In [28]:
from sklearn.ensemble import BaggingClassifier
clf = tree.DecisionTreeClassifier(criterion='entropy')

bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = bagging_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9649122807017544

When I did this using a single decision tree classifier was 90.3% accurate, while the bagging classifier was 96.5% accurate--halving the error rate! that's pretty good!


### Pasting
Let's try the same thing with pasting (without replacement):

For that we set the hyperparameter: `bootstrap=False`


In [29]:
pasting_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=False, n_jobs=-1)
pasting_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = pasting_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9736842105263158

### Random Subspaces
Again, random subspaces are when we randomly select feature subsets rather than subsets of the dataset instances. This time we will create 50 classifiers for our collective and each will train on a dataset with 7 features (`max_feature=7`).

In [30]:
subspace_clf = BaggingClassifier(clf, n_estimators=50, max_features=7, 
                                bootstrap=True, n_jobs=-1)
subspace_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = subspace_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9649122807017544

### Random Patches
Finally, let's combine things and try random patches. In this example each classifier will be given a subset of 100 training instances with 7 features each.:

In [31]:
subspace_clf = BaggingClassifier(clf, n_estimators=100, max_features=7, 
                                 max_samples=100, bootstrap=False, n_jobs=-1)
subspace_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = subspace_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9649122807017544

While it is common to use a decision tree as the base classifier, we can use any classifier. Here we use kNN:

In [32]:
from sklearn.neighbors import KNeighborsClassifier
kNN = KNeighborsClassifier()
bagging_clf = BaggingClassifier(kNN, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(trainingDataFeatures, trainingDataLabels)
predictions = bagging_clf.predict(testDataFeatures)
accuracy_score(testDataLabels, predictions)

0.9298245614035088

#### Summary
As you can see, any of these simple bagging algorithms typically outperforms using a single classifier. 


### Review

We import the bagging classifier library with:

```
from sklearn.ensemble import BaggingClassifier
```

and create an instance of one with:

```
my_bagging_classifier = BaggingClassifier(baseClassifier, Hyperparameters,n_jobs=-1)
```

#### Base Classifier
while any classifier can be used we typically use a decision tree

#### Hyperparameters
Here is a list of the hyperparameters (from the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)):

* `n_estimators`: integer, default value = 10, the number of classifiers (estimators) in the ensemble.
* `max_samples`: integer or float, default value = 1.0(meaning use all the training instances), the number of samples (instances) to draw from the training dataset to train each base classifier.
    * if integer, then draw max_features features.
    * if float, then draw max_samples * X.shape[0] samples. For example if `max_samples` is 0.7 and there are 100 instances in the training dataset then draw 70 samples.
* `max_features`, integer or float, default value =1.0,  
the number of features to draw from the training dataset to train each base estimator 
    * if integers, then draw max_features features.
    * if float, then draw max_features * X.shape[1] features.
* `bootstrap` boolean, default value =True, whether samples and features are drawn with replacement. If False, sampling without replacement is performed.

For other hyperparamters, consult the documentation.


![](https://raw.githubusercontent.com/zacharski/datamining-guide/master/labs/pics/torchdivide.png)


# <font color='#EE4C2C'>You Try ...</font> 
## <font color='#EE4C2C'>Predicting musical genres from audio file attributes</font> 


When you listen even to a few seconds of a song you can identify it as blues, country, classical, or any other genre. How do you do this? What attributes are you hearing in the audio file that helps you make this classification? And, more to the point, can we train a computer to do it?

![](https://raw.githubusercontent.com/zacharski/ml-class/master/labs/pics/bluesClassical.png)

We are going to be using  the [GTZAN Dataset for Music Genre Classification](https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification). It provides data of 100 songs for each of 10 genres. The data is in several formats:

* 30 second audio files (wav)
* spectral images of those 30 second clips (see image above)
* a csv file containing acoustic attributes of the 30 second clip
* a csv file containing acoustic attributes of 3 second clips (the 30 second clips were split into 3 second ones)

We are going to use the 3 second csv file which is available at 

https://raw.githubusercontent.com/zacharski/ml-class/master/data/gtzan.csv

Go ahead and load the data into a dataframe (the first row contains feature names)


In [33]:
music = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/gtzan.csv')

Let's examine the values of the label column (the genres):

In [34]:
music.label.unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

Those are the 10 genres we are trying to predict. So if we were just to guess without hearing the clip, we would be accurate 10% of the time. How accurate do you think you would be based on hearing a 3 second clip? I am pretty confident I could correctly label the 30 second clips, but I am much less confident about labeling 3 second ones. Since guessing randomly would give me 10% accuracy, I am estimating maybe 50-60% accuracy. Let's see how a computer does.

#### Feature Names
So the column we are trying to predict is `label`. Now let's get the names of the feature columns

In [35]:
featureNames = list(music.columns)
featureNames.remove('filename')
featureNames.remove('label')
print(featureNames)
print(len(featureNames))

['length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var', 'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean', 'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var', 'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var']
58


So we have 58 features. 

#### Training and test sets
Now it is time to construct the training and test sets:

In [42]:
music_training, music_test = train_test_split(music, test_size = 0.2)

list(music_training.columns)
                              
## now create the DataFrames for just the features (excluding the label column 
## filename column)                              
music_training_features = music_training[featureNames]
music_test_features = music_test[featureNames]
                              
## now create the labels data structure for both the training and test sets                              
music_training_labels = music_training['label']
music_test_labels = music_test['label']
music_test_labels

6082      metal
7604        pop
4650     hiphop
7309        pop
2476    country
         ...   
2039    country
5938       jazz
4275     hiphop
7023        pop
2983    country
Name: label, Length: 1998, dtype: object

### Building a single decision tree classifier
Let's build a single decision tree classifier called `clf` using entropy, fit it to the data, make predictions and determine the accuracy:

In [37]:
## Create clf, an instance of the Decision Tree Classifier
music_clf = tree.DecisionTreeClassifier(criterion='entropy')

## Fit it to the data
music_clf.fit(music_training_features, music_training_labels)

## get the predictions for the test set
music_predictions = music_clf.predict(music_test_features)

## get the accuracy score
accuracy_score(music_test_labels, music_predictions)

0.6721721721721722

When I did this I got 66% accuracy. That doesn't sound great but keep in mind that random guessing would only be 10% accuracy. 

### Building a Random Patch Classifier

Now we are going to build a random patch classifier.

* the base classifier will be a decision tree using entropy
* the ensemble will contain 20 base classifiers
* each classifier will use a random sample of 70% of the training data
* each classifier will use a random sample of 70% of the features
* the sampling will be done with replacement
* it will use all available cpu cores.

We are going to

1. build the classifier
2. train the classifier on the data
3. make predictions on the test set
4. determine the accuracy

In [38]:
music_random_patch_clf = BaggingClassifier(music_clf, n_estimators=20, max_features=0.7, 
                                 max_samples=0.7, bootstrap=True, n_jobs=-1)
music_random_patch_clf.fit(music_training_features, music_training_labels)
music_random_patch_predictions = music_random_patch_clf.predict(music_test_features)
accuracy_score(music_test_labels, music_random_patch_predictions)

0.8333333333333334

What accuracy did you get? Was it better than using a single classifier?
Keep your original code above. Make a copy of it below and 
experiment a bit with the hyperparameters. (try 3 or 4 different things) What is the best accuracy you can get?

In [39]:
#0.8253253253253253

In [40]:
music_random_patch_clf = BaggingClassifier(music_clf, n_estimators=20, max_features=0.8, 
                                 max_samples=0.8, bootstrap=False, n_jobs=-1)
music_random_patch_clf.fit(music_training_features, music_training_labels)
music_random_patch_predictions = music_random_patch_clf.predict(music_test_features)
accuracy_score(music_test_labels, music_random_patch_predictions)

0.8538538538538538

In [41]:
#0.8433433433433434


##<font color='#52BE80'>NOTES</font>

### Breast Cancer Database

[back](#Wisconsin-Cancer-Dataset)

  This breast cancer databases was obtained from the University of Wisconsin
   Hospitals, Madison from Dr. William H. Wolberg.  If you publish results
   when using this database, then please include this information in your
   acknowledgements.  Also, please cite one or more of:

   1. O. L. Mangasarian and W. H. Wolberg: "Cancer diagnosis via linear 
      programming", SIAM News, Volume 23, Number 5, September 1990, pp 1 & 18.

   2. William H. Wolberg and O.L. Mangasarian: "Multisurface method of 
      pattern separation for medical diagnosis applied to breast cytology", 
      Proceedings of the National Academy of Sciences, U.S.A., Volume 87, 
      December 1990, pp 9193-9196.

   3. O. L. Mangasarian, R. Setiono, and W.H. Wolberg: "Pattern recognition 
      via linear programming: Theory and application to medical diagnosis", 
      in: "Large-scale numerical optimization", Thomas F. Coleman and Yuying
      Li, editors, SIAM Publications, Philadelphia 1990, pp 22-30.

   4. K. P. Bennett & O. L. Mangasarian: "Robust linear programming 
      discrimination of two linearly inseparable sets", Optimization Methods
      and Software 1, 1992, 23-34 (Gordon & Breach Science Publishers).

Title: Wisconsin Breast Cancer Database (January 8, 1991)


Sources:
   -- Dr. WIlliam H. Wolberg (physician)
      University of Wisconsin Hospitals
      Madison, Wisconsin
      USA
   -- Donor: Olvi Mangasarian (mangasarian@cs.wisc.edu)
      Received by David W. Aha (aha@cs.jhu.edu)
   -- Date: 15 July 1992